In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'].insert(0, 'Malgun Gothic')
sns.set(font="Malgun Gothic", rc = {"axes.unicode_minus":False}, style='whitegrid')
plt.rcParams["figure.figsize"] = (15, 6)

from tqdm import notebook
import time

In [2]:
filename = list()
for f in os.listdir('../Data'):
    filename.append(f)
filename

['raw_data']

In [3]:
DEMO = pd.read_csv(os.path.join('../Data', filename[0])) #고객데모정보
PDDE = pd.read_csv(os.path.join('../Data', filename[1])) #유통사 상품 구매 내역
COP_U = pd.read_csv(os.path.join('../Data', filename[2])) #제휴사 서비스 이용 내역
PD_CLAC = pd.read_csv(os.path.join('../Data', filename[3])) #유통사 상품 카테고리 마스터
BR = pd.read_csv(os.path.join('../Data', filename[4])) #유통사/제휴사 점포 마스터
LPAY = pd.read_csv(os.path.join('../Data', filename[5])) #엘페이 결제 내역

PermissionError: [Errno 13] Permission denied: '../Data\\raw_data'

In [ ]:
DEMO.rename(columns={'cust':'고객번호', 'ma_fem_dv':'성별', 'ages':'연령대', 'zon_hlv':'거주지'}, inplace=True)
PDDE.rename(columns={'cust':'고객번호', 'rct_no':'영수증', 'chnl_dv':'채널', 'cop_c':'제휴사', 'pd_c':'상품코드', 'br_c':'점포코드', 'de_dt':'이용일자', 'de_hr':'이용시간', 'buy_am':'이용금액', 'buy_ct':'구매수량'}, inplace=True)
COP_U.rename(columns={'cust':'고객번호', 'rct_no':'영수증', 'cop_c':'제휴사', 'br_c':'점포코드', 'de_dt':'이용일자', 'vst_dt':'방문일자', 'de_hr':'이용시간', 'buy_am':'이용금액', 'chnl_dv':'채널'}, inplace=True)
PD_CLAC.rename(columns={'pd_c':'상품코드', 'pd_nm':'소분류', 'clac_hlv_nm':'대분류', 'clac_mcls_nm':'중분류'}, inplace=True)
BR.rename(columns={'br_c':'점포_코드', 'cop_c':'제휴사', 'zon_hlv':'점포_대분류', 'zon_mcls':'점포_중분류', 'chnl_dv':'채널'}, inplace=True)
LPAY.rename(columns={'cust':'고객번호', 'cop_c':'제휴사', 'rct_no':'영수증', 'de_dt':'이용일자', 'de_hr':'이용시간', 'buy_am':'이용금액', 'chnl_dv':'채널'}, inplace=True)

In [ ]:
COP_U_D = COP_U[(COP_U['제휴사']=='D01') | (COP_U['제휴사']=='D02')]
BR_D = BR[(BR['제휴사']=='D01') | (BR['제휴사']=='D02')]
LPAY_D = LPAY[(LPAY['제휴사']=='D01') | (LPAY['제휴사']=='D02')]

## 데이터 살펴보자

In [ ]:
DEMO #1. 고객데모정보

In [ ]:
PDDE #2. 유통사 상품 구매 내역

In [ ]:
COP_U

In [ ]:
len('A01 0 5920171 5')#1 DEMO

In [ ]:
len('211010 09 D01 5920171') 
#3

In [ ]:
len('210803 21 03 11226')#6 LPAY

In [ ]:
LPAY

In [ ]:
PDDE[PDDE['영수증'].str.contains('5920171')]

In [ ]:
DEMO[DEMO['고객번호']=='M848023775']

In [ ]:
LPAY_D['영수증'] = LPAY_D['영수증'].astype('str')
LPAY_D[LPAY_D['영수증'].str.contains('5920171')]

In [ ]:
LPAY_D[LPAY_D['고객번호'].str.contains('M848023775')]

In [ ]:
COP_U_D #3. 제휴사 서비스 이용 내역

In [ ]:
PD_CLAC #4. 유통사 상품 카테고리 마스터

In [ ]:
print('-----------대분류-----------\n', set(PD_CLAC['대분류']))
print('\n-----------중분류-----------\n', set(PD_CLAC['중분류']))
print('\n-----------소분류-----------\n', set(PD_CLAC['소분류']))

미쳤나 개많네

In [ ]:
BR_D #5. 유통사/제휴사 점포 마스터

In [ ]:
print('-----------점포_대분류-----------\n', sorted(set(BR_D['점포_대분류'])))
print('\n-----------점포_중분류-----------\n', sorted(set(BR_D['점포_중분류'])))

In [ ]:
LPAY_D #6.엘페이 결제 내역

## 제휴사 이용 정보 + 고객데모정보

In [ ]:
COP_U_D2 = COP_U_D[]

In [ ]:
df2 = pd.concat(COP_U_D, LPAY_D, how='left',on='영수증')
df2

In [ ]:
df = pd.merge(COP_U_D, DEMO, how='left',on='고객번호')
df

In [ ]:
set(df['연령대'])

In [ ]:
df['연령대'] = df['연령대'].apply(lambda x: int(x[:-1]))
set(df['연령대'])

In [ ]:
df[df['이용일자']!=df['방문일자']]

이용일자와 방문일자가 다른 경우는 없었고,,

In [ ]:
#이용일자를 월/분기별로
df['이용월'] = df['이용일자'].apply(lambda x:int(str(x)[4:6]))
df['분기'] = df['이용일자'].apply(lambda x: (int(str(x)[4:6])//3) if (int(str(x)[4:6])%3)==0 else (int(str(x)[4:6])//3) + 1)

## 상관도를 알아보자

In [ ]:
heatmap_df = df[['채널', '분기', '이용금액', '성별']]
heatmap_df['성별'] = heatmap_df['성별'].apply(lambda x: 1 if x=='남성' else 2)

In [ ]:
#연령대를 dummy변수로
heatmap_df.loc[:, '20대'] = df['연령대']
heatmap_df.loc[:, '30대'] = df['연령대']
heatmap_df.loc[:, '40대'] = df['연령대']
heatmap_df.loc[:, '50대'] = df['연령대']
heatmap_df.loc[:, '60대'] = df['연령대']
heatmap_df.loc[:, '70대'] = df['연령대']
heatmap_df.replace({'20대':{20:1, 30:0, 40:0, 50:0, 60:0, 70:0},
                   '30대':{20:0, 30:1, 40:0, 50:0, 60:0, 70:0},
                   '40대':{20:0, 30:0, 40:1, 50:0, 60:0, 70:0},
                   '50대':{20:0, 30:0, 40:0, 50:1, 60:0, 70:0},
                   '60대':{20:0, 30:0, 40:0, 50:0, 60:1, 70:0},
                   '70대':{20:0, 30:0, 40:0, 50:0, 60:0, 70:1}}, inplace = True)

In [ ]:
heatmap_df

In [ ]:
heatmap_df.astype(float).corr()

In [ ]:
#colormap = plt.cm.PuBu 
plt.figure(figsize=(10, 10)) 
plt.title("Correlation", y = 1.05, size = 15) 
sns.heatmap(heatmap_df.astype(float).corr() , linewidths = 0.1, vmax = 1.0, 
            square = True, linecolor = "white", annot = True, annot_kws = {"size" : 12})


머 이런 쓰레기상관도가...

In [ ]:
heatmap_df2 = heatmap_df.copy()
heatmap_df2['이용금액'] = np.log(heatmap_df2['이용금액'])

In [ ]:
heatmap_df2

In [ ]:
#colormap = plt.cm.PuBu 
plt.figure(figsize=(10, 10)) 
plt.title("Correlation", y = 1.05, size = 15) 
sns.heatmap(heatmap_df2.astype(float).corr() , linewidths = 0.1, vmax = 1.0, 
            square = True, linecolor = "white", annot = True, annot_kws = {"size" : 12})

아 일단 넘어간다

## 그래프 그리자

In [ ]:
df.columns

In [ ]:
# 이용금액 단위 변경 (천원)
df['이용금액'] = df['이용금액']/1000

### 월별 이용 데이터

In [ ]:
df_월별_수= df[['이용월']].groupby(['이용월'], as_index=False).size()
df_월별_수

In [ ]:
sum(df_월별_수['size'])

In [ ]:
len(df)

In [ ]:
#월별 이용 확인
df_월별_합 = df[['이용월', '이용금액']].groupby(['이용월'], as_index=False).sum()
df_월별_평균 = df[['이용월', '이용금액']].groupby(['이용월'], as_index=False).mean()
df_월별_수= df[['이용월']].groupby(['이용월'], as_index=False).size()

df_월별_합.plot(x='이용월', y='이용금액', kind='bar', title='월별 총 이용금액', color = sns.color_palette('PuRd'))
df_월별_평균.plot(x='이용월', y='이용금액', kind='bar', title='월별 평균 이용금액', color = sns.color_palette('PuRd'))
df_월별_수.plot(x='이용월', y='size', kind='bar', title='월별 이용 빈도', color = sns.color_palette('PuRd'))

### 분기별 이용데이터

In [ ]:
#분기별 이용 확인
df_분기별_합 = df[['분기', '이용금액']].groupby(['분기'], as_index=False).sum()
df_분기별_평균 = df[['분기', '이용금액']].groupby(['분기'], as_index=False).mean()
df_분기별_수= df[['분기']].groupby(['분기'], as_index=False).size()

df_분기별_합.plot(x='분기', y='이용금액', kind='bar', title='분기별 총 이용금액', color = sns.color_palette('PuRd'))
df_분기별_평균.plot(x='분기', y='이용금액', kind='bar', title='분기별 평균 이용금액', color = sns.color_palette('PuRd'))
df_분기별_수.plot(x='분기', y='size', kind='bar', title='분기별 이용 빈도', color = sns.color_palette('PuRd'))

## 채널별

In [ ]:
df_채널별_합 = df[['채널', '이용금액']].groupby(['채널'], as_index=False).sum()
df_채널별_평균 = df[['채널', '이용금액']].groupby(['채널'], as_index=False).mean()
df_채널별_수= df[['채널']].groupby(['채널'], as_index=False).size()

df_채널별_합.plot(x='채널', y='이용금액', kind='bar', title='채널별 총 이용금액', color = sns.color_palette('PuRd'))
df_채널별_평균.plot(x='채널', y='이용금액', kind='bar', title='채널별 평균 이용금액', color = sns.color_palette('PuRd'))
df_채널별_수.plot(x='채널', y='size', kind='bar', title='채널별 이용 빈도', color = sns.color_palette('PuRd'))

채널이 다 online뿐,, 그냥 offline 없이 하자~~~~

## 이용시간별

In [ ]:
df_이용시간별_합 = df[['이용시간', '이용금액']].groupby(['이용시간'], as_index=False).sum()
#df_이용시간별_평균 = df[['이용시간', '이용금액']].groupby(['이용시간'], as_index=False).mean()
df_이용시간별_수= df[['이용시간']].groupby(['이용시간'], as_index=False).size()

df_이용시간별_합.plot(x='이용시간', y='이용금액', kind='bar', title='이용시간별 총 이용금액', color = sns.color_palette('PuRd'))
#df_이용시간별_평균.plot(x='이용시간', y='이용금액', kind='bar', title='이용시간별 평균 이용금액', color = sns.color_palette('PuRd'))
df_이용시간별_수.plot(x='이용시간', y='size', kind='bar', title='이용시간별 이용 빈도', color = sns.color_palette('PuRd'))

### 연령대별

In [ ]:
heatmap_df = df[['연령대', '이용시간', '이용금액']]
heatmap_df
#colormap = plt.cm.PuBu 
plt.figure(figsize=(10, 10)) 
plt.title("Correlation", y = 1.05, size = 15) 
sns.heatmap(heatmap_df.astype(float).corr() , linewidths = 0.1, vmax = 1.0, 
            square = True, linecolor = "white", annot = True, annot_kws = {"size" : 12})

In [ ]:
df

In [ ]:
df_연령대별_합 = df[['연령대', '이용금액']].groupby(['연령대'], as_index=False).sum()
#df_연령대별_평균 = df[['연령대', '이용금액']].groupby(['연령대'], as_index=False).mean()
#df_연령대별_수= df[['연령대']].groupby(['연령대'], as_index=False).size()

df_연령대별_합.plot(x='연령대', y='이용금액', kind='bar', title='연령대별 총 이용금액', color = sns.color_palette('PuRd'))
#df_연령대별_평균.plot(x='연령대', y='이용금액', kind='bar', title='연령대별 평균 이용금액', color = sns.color_palette('PuRd'))
#df_연령대별_수.plot(x='연령대', y='size', kind='bar', title='연령대별 이용 빈도', color = sns.color_palette('PuRd'))

In [ ]:
#연령대별 시간대
df_연령대_시간 = df[['연령대', '이용시간', '이용금액']].groupby(['연령대','이용시간'], as_index=False).sum()
df_연령대_시간

In [ ]:
df_연령대_시간[df_연령대_시간['연령대']==20].plot(x='이용시간', y='이용금액', kind='bar', title='20대 총 이용금액', color = sns.color_palette('PuRd'))
df_연령대_시간[df_연령대_시간['연령대']==30].plot(x='이용시간', y='이용금액', kind='bar', title='30대 총 이용금액', color = sns.color_palette('PuRd'))
df_연령대_시간[df_연령대_시간['연령대']==40].plot(x='이용시간', y='이용금액', kind='bar', title='40대 총 이용금액', color = sns.color_palette('PuRd'))
df_연령대_시간[df_연령대_시간['연령대']==50].plot(x='이용시간', y='이용금액', kind='bar', title='50대 총 이용금액', color = sns.color_palette('PuRd'))
df_연령대_시간[df_연령대_시간['연령대']==60].plot(x='이용시간', y='이용금액', kind='bar', title='60대 총 이용금액', color = sns.color_palette('PuRd'))
df_연령대_시간[df_연령대_시간['연령대']==70].plot(x='이용시간', y='이용금액', kind='bar', title='70대 총 이용금액', color = sns.color_palette('PuRd'))

## 거주지별

In [ ]:
df_거주지별_합 = df[['거주지', '이용금액']].groupby(['거주지'], as_index=False).sum()
#df_연령대별_평균 = df[['연령대', '이용금액']].groupby(['연령대'], as_index=False).mean()
df_거주지별_수= df[['거주지']].groupby(['거주지'], as_index=False).size()

df_거주지별_합.plot(x='거주지', y='이용금액', kind='bar', title='거주지별 총 이용금액', color = sns.color_palette('PuRd'))
#df_연령대별_평균.plot(x='연령대', y='이용금액', kind='bar', title='연령대별 평균 이용금액', color = sns.color_palette('PuRd'))
df_거주지별_수.plot(x='거주지', y='size', kind='bar', title='거주지별 이용 빈도', color = sns.color_palette('PuRd'))

In [ ]:
df_거주지별_합.sort_values('이용금액', ascending=False)

In [ ]:
custom_palette = sns.color_palette("Set3", 6)
sns.histplot(data=df, x='거주지', hue='연령대', palette = custom_palette)

In [ ]:
\=


# 해킹중

In [ ]:
DEMO.head() # 고객정보

In [ ]:
PDDE.head() # 2.유통사 

In [ ]:
COP_U.head() #3제휴사

In [ ]:
PD_CLAC.head() #4

In [ ]:
BR.head()

In [ ]:
LPAY.head()

In [ ]:
df = pd.merge(COP_U, DEMO)
len(df)==len(COP_U)
df.head()

In [ ]:
df['일련번호'] = df['고객번호']+df['이용일자'].astype('str')+df['이용시간'].astype('str')+df['이용금액'].astype('str')

In [ ]:
df

In [ ]:
LPAY

In [ ]:
LPAY['일련번호'] = LPAY['고객번호']+LPAY['이용일자'].astype('str')+LPAY['이용시간'].astype('str')+LPAY['이용금액'].astype('str')

In [ ]:
df2 = LPAY.copy()

In [ ]:
df

In [ ]:
df2

In [ ]:
pd3=pd.merge(df[['일련번호', '이용금액']], df2[['일련번호', '이용금액']], on='일련번호')
pd3

In [ ]:
len(df)

In [ ]:
pd3[pd3['이용금액_x']!=pd3['이용금액_y']]

In [ ]:
df[df['일련번호']=='M0280649532021091318']
df2[df2['일련번호']=='M0280649532021091318']

In [ ]:
df2[df2['일련번호']=='M0280649532021091318']

In [ ]:
display(df[df['일련번호']=='M2748270572021121316'], df2[df2['일련번호']=='M2748270572021121316'])

In [ ]:
df

In [ ]:
df2

In [ ]:
PD_CLAC